In [ ]:
 

# def assessor(img_list, layout=(2, 2), threshold=175, h=19):
#     """
#     Compare effectiveness of specific parameterization on test images
#     by visual comparison.
#     """
#     MARK_COLOR  = (0, 0, 255)
#     MARKER_TYPE = cv2.MARKER_SQUARE # cv2.MARKER_DIAMOND cv2.MARKER_TRIANGLE_UP cv2.MARKER_STAR cv2.MARKER_CROSS
#     figsize     = (11, 5)
#     dpi         = 100
#     cmap        = plt.cm.gray
#     images      = [img.copy() for img in img_list]
#     ax_indx     = list(itertools.product(range(layout[0]), range(layout[-1])))
    
#     disp_images = []
#     for img in images:
#         img_ = img.copy()
#         mono = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
        
#         BLOCKSIZE   = 2
#         KSIZE       = 3
#         K           = .06
#         # blur        = cv2.GaussianBlur(mono, (3, 3), 0)
#         # blur       = cv2.medianBlur(mono, 5)
#         blur        = cv2.fastNlMeansDenoising(mono, h=h, templateWindowSize=7, searchWindowSize=21).astype(np.float32)
#         # blur_spec   = "gaussian(5, 5)" # "median(3)" 
        
        
#         harris = cv2.cornerHarris(blur, blockSize=BLOCKSIZE, ksize=KSIZE, k=K)
#         temp = np.empty(harris.shape, dtype=np.float32)
#         cv2.normalize(harris, temp, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
#         harris_scl = cv2.convertScaleAbs(temp)
#         indx_ = np.where(harris_scl>threshold)
#         indx_arr = np.hstack([np.asarray(aa, dtype=np.float32).reshape(-1,1) for aa in indx_])
#         criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
#         _, label, means = cv2.kmeans(indx_arr, 4, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
#         grps_list = [indx_arr[label.ravel()==ii] for ii in range(4)]
#         for ii, jj in means:
#             cv2.drawMarker(
#                 img_, (int(jj), int(ii)), MARK_COLOR, markerType=MARKER_TYPE, markerSize=3, thickness=2
#                 )
#         disp_images.append(img_)
        
            
#     # Display images.
#     fig, ax = plt.subplots(layout[0], layout[-1], figsize=figsize, dpi=150, tight_layout=True)
#     title_str = "blocksize={}, ksize={}, k={}, threshold={}, h={}".format(BLOCKSIZE, KSIZE, K, threshold, h)
#     fig.suptitle(title_str, color="#FF0000", size=8)
    
#     for rc, img__ in itertools.zip_longest(ax_indx, disp_images):
#         r_indx, c_indx = rc
#         if layout[0]==1 and layout[-1]==1:
#             ax_ii = ax
#         elif layout[0]==1 and layout[-1]>1:
#             ax_ii = ax[c_indx]
#         else:
#             ax_ii = ax[r_indx, c_indx]
        
#         # Remove facet if img_ is None.
#         if img is None: ax_ii.axis("off")
#         ax_ii.imshow(b2r(img__), cmap=cmap)
#         ax_ii.set_axis_off()
        
#     plt.show()    



def assessor(img_list, layout=(3, 2), threshold=.80):
    """
    Compare effectiveness of specific parameterization on test images
    by visual comparison.
    """
    MARK_COLOR  = (255, 0, 255)
    MARKER_TYPE = cv2.MARKER_CROSS #cv2.MARKER_SQUARE # cv2.MARKER_DIAMOND cv2.MARKER_TRIANGLE_UP cv2.MARKER_STAR cv2.MARKER_CROSS
    figsize     = (9, 6)
    dpi         = 125
    cmap        = plt.cm.gray
    images      = [img.copy() for img in img_list]
    ax_indx     = list(itertools.product(range(layout[0]), range(layout[-1])))
    tmpl_mono   = cv2.cvtColor(tmpl, cv2.COLOR_BGR2GRAY)
    h, w        = tmpl_mono.shape
    
    disp_images = []
    for img in images:
        img_ = img.copy()
        mono = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
        temp = cv2.matchTemplate(mono, tmpl_mono, cv2.TM_CCOEFF_NORMED)
        indx_init = np.where(temp>=threshold)
        indx_arr = np.hstack([np.asarray(aa, dtype=np.float32).reshape(-1,1) for aa in indx_init])
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        _, labels, means = cv2.kmeans(indx_arr, 4, None, criteria, 10, cv2.KMEANS_PP_CENTERS)

        # Reverse x and y indices. 
        x, y = list(zip(*indx_arr))
        rev_arr = np.asarray(list(zip(y, x)))
        grps_list = [rev_arr[labels.ravel()==ii] for ii in range(4)]
        
        centers_list = []
        for ii, grp in enumerate(grps_list):
            v1_init = grps_list[ii].mean(axis=0)
            v1 = np.asarray([int(ii) for ii in v1_init])
            v2 = np.asarray([v1[0] + w, v1[-1] + h])
            pt1, pt2 = tuple([int(ii) for ii in v1]), tuple([int(ii) for ii in v2])

            # Get distance to center of circle. 
            dx, dy = pt2[0] - pt1[0], pt2[-1] - pt1[-1]
            dxh, dyh = dx // 2, dy // 2
            center_x, center_y = pt1[0] + dxh, pt1[-1] + dyh
            centers_list.append((center_x, center_y))

            # Draw locations of center points on original image. 
            cv2.drawMarker(
                img_, (center_x, center_y), MARK_COLOR, markerType=MARKER_TYPE, markerSize=13, thickness=2
                )
            cv2.putText(
                img_, "[{},{}]".format(center_x, center_y), (center_x+15, center_y-15), 
                cv2.FONT_HERSHEY_SIMPLEX, .35, (0, 0, 0), 1
                )
        disp_images.append(img_)

    print(centers_list)
    # Display images.
    fig, ax = plt.subplots(layout[0], layout[-1], figsize=figsize, dpi=dpi, tight_layout=True)
    title_str = "threshold={}".format(threshold)
    fig.suptitle(title_str, color="#FF0000", size=8)
    
    for rc, img__ in itertools.zip_longest(ax_indx, disp_images):
        r_indx, c_indx = rc
        if layout[0]==1 and layout[-1]==1:
            ax_ii = ax
        elif layout[0]==1 and layout[-1]>1:
            ax_ii = ax[c_indx]
        else:
            ax_ii = ax[r_indx, c_indx]
        
        # Remove facet if img_ is None.
        if img is None: 
            ax_ii.axis("off")
        else:
            ax_ii.imshow(b2r(img__), cmap=cmap)
            ax_ii.set_axis_off()
    plt.show()   

    
imglist = [img0, img1, img2, img3, img4, img5,]   
assessor(imglist, layout=(3, 2), threshold=.80)
